In [1]:
#do this later from admin: !python -m pip install --upgrade pip


In [1]:
#Basic
import pandas as pd
import numpy as np
#Math
import scipy as sp
from random import randint
from math import exp
import operator

#Scraping
import requests
import json

#API Packages: 
from apiclient.discovery import build
from apiclient.errors import HttpError
from oauth2client.tools import argparser
"""import argparse
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError"""

#Modeling
#from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV
#from sklearn.linear_model import LogisticRegression
#from sklearn.naive_bayes import MultinomialNB
#from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, BaggingClassifier
#from sklearn.neighbors import KNeighborsClassifier
#from sklearn.cluster import KMeans
#from sklearn.svm import SVC
#from sklearn.metrics import confusion_matrix, roc_auc_score, roc_curve

#NLP Processing
'''from sklearn.feature_extraction import stop_words
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer'''

#Time analysis
import time
import datetime
from statsmodels.tsa.stattools import pacf
from statsmodels.graphics.tsaplots import plot_pacf

#Plotting
import matplotlib.pyplot as plt
import seaborn as sns
#from wordcloud import WordCloud

%matplotlib inline
plt.style.use('seaborn-whitegrid')

In [2]:
my_api_file = open("./mykey.txt", 'r')
my_api_str = my_api_file.read()
my_api_file.close()

In [3]:
DEVELOPER_KEY = my_api_str
YOUTUBE_API_SERVICE_NAME = "youtube"
YOUTUBE_API_VERSION = "v3"

In [ ]:
# all views over 400M https://www.youtube.com/playlist?list=PLirAqAtl_h2r5g8xGajEwdXd3x1sZh8hC

https://www.youtube.com/results?filters=video&lclk=video&search_query=a%7Cb%7Cc%7Cd%7Ce%7Cf%7Cg%7Ch%7Ci%7Cj%7Ck%7Cl%7Cm%7Cn%7Co%7Cp%7Cq%7Cr%7Cs%7Ct%7Cu%7Cv%7Cw%7Cx%7Cy%7Cz%7C1%7C1%7C2%7C3%7C4%7C5%7C6%7C7%7C8%7C9%7C0&search_sort=video_view_count


YouTube Search_list filtering parameters (sort_method):
* date – Resources are sorted in reverse chronological order based on the date they were created.
* rating – Resources are sorted from highest to lowest rating.
* relevance – Resources are sorted based on their relevance to the search query. This is the default value for this parameter.
* title – Resources are sorted alphabetically by title.
* videoCount – Channels are sorted in descending order of their number of uploaded videos.
* viewCount – Resources are sorted from highest to lowest number of views. For live broadcasts, videos are sorted by number of concurrent viewers while the broadcasts are ongoing.

In [4]:
#This function returns a list of videos according to search query 'q' as if you were typing it into the bar
#The out put is a json list, predominantly used to get videoId's
def youtube_search_list(q, max_results=50, sort_method='viewCount', token=None):
  # Call the search.list method to retrieve results matching the specified query term.
    youtube = build(YOUTUBE_API_SERVICE_NAME, YOUTUBE_API_VERSION,
    developerKey=DEVELOPER_KEY)

  # Call the search.list method to retrieve results matching the specified query term.
    search_response = youtube.search().list(
        q=q,
        part='id,snippet',
        pageToken=token,
        maxResults=max_results,
        order=sort_method
      ).execute()
    
    #output is called listresponse in api docs
    return search_response

In [10]:
youtubez = build(YOUTUBE_API_SERVICE_NAME, YOUTUBE_API_VERSION, developerKey=DEVELOPER_KEY)
youtubez.search().list(
    part='id,snippet',
    q='a', 
    videoCategoryId='10',
    type='video'
    
      ).execute()

{'kind': 'youtube#searchListResponse',
 'etag': '"XI7nbFXulYBIpL0ayR_gDh3eu1k/0Q3nkSI7oRm8zuPxi4Fv-UG_BEo"',
 'nextPageToken': 'CAUQAA',
 'regionCode': 'US',
 'pageInfo': {'totalResults': 1000000, 'resultsPerPage': 5},
 'items': [{'kind': 'youtube#searchResult',
   'etag': '"XI7nbFXulYBIpL0ayR_gDh3eu1k/K8_MNnp69L0zrHbPM_-nbwUCGaY"',
   'id': {'kind': 'youtube#video', 'videoId': '1tAgFLmM0UU'},
   'snippet': {'publishedAt': '2018-09-25T14:56:58.000Z',
    'channelId': 'UC6rwiIxv0w2fbmmr66wl1rA',
    'title': 'Luan Santana | "A" (Video Oficial) - Live-Móvel',
    'description': 'Ouça #A e todo o EP #LiveMovel https://SomLivre.lnk.to/Live-Movel Composição: Bruno Caliman / #LuanSantana / Lucas Santos / Rafael Torres Tá em dúvida ...',
    'thumbnails': {'default': {'url': 'https://i.ytimg.com/vi/1tAgFLmM0UU/default.jpg',
      'width': 120,
      'height': 90},
     'medium': {'url': 'https://i.ytimg.com/vi/1tAgFLmM0UU/mqdefault.jpg',
      'width': 320,
      'height': 180},
     'high': 

In [5]:
#takes output json list from search list and returns metadata on individual videos
def youtube_search_video(q='spinners', max_results=50, sort_method='viewCount', token=None):
    
    order = "viewCount"
    q=q
    max_results = max_results
    sort_method = sort_method
    token = token
    location = None
    location_radius = None
    
    youtube = build(YOUTUBE_API_SERVICE_NAME, YOUTUBE_API_VERSION,         
    developerKey=DEVELOPER_KEY)
    
    #Return list of matching records up to max_search
    search_result = youtube_search_list(q, max_results, sort_method, token)
    next_tok = search_result['nextPageToken']
    
    videos_list = []
    for search_result in search_result.get("items", []):
        
        if search_result["id"]["kind"] == 'youtube#video':
            temp_dict_ = {}
            #Available from initial search
            temp_dict_['title'] = search_result['snippet']['title']  
            temp_dict_['vidId'] = search_result['id']['videoId']  
            
            #Secondary call to find statistics results for individual video
            response = youtube.videos().list(
                part='statistics, snippet, contentDetails', 
                id=search_result['id']['videoId']
                    ).execute()
            #Relevant dictionaries
            response_statistics = response['items'][0]['statistics']
            response_snippet = response['items'][0]['snippet']
            response_content= response['items'][0]['contentDetails']
            
            
            snippet_list = ['publishedAt','channelId', 'description', 
                            'channelTitle', 'tags', 'categoryId', 
                            'liveBroadcastContent', 'defaultLanguage', ]
            for val in snippet_list:
                try:
                    temp_dict_[val] = response_snippet[val]
                except:
                    #Not stored if not present
                    temp_dict_[val] = 'xxNoneFoundxx'    
            
            stats_list = ['favoriteCount', 'viewCount', 'likeCount', 
                          'dislikeCount', 'commentCount']
            for val in stats_list:
                try:
                    temp_dict_[val] = response_statistics[val]
                except:
                    #Not stored if not present
                    temp_dict_[val] = 'xxNoneFoundxx'
            
            for val in response_content.keys():
                try:
                    temp_dict_[val] = response_content[val]
                except:
                    #Not stored if not present
                    temp_dict_[val] = 'xxNoneFoundxx'
                    
            #add back to main list
            videos_list.append(temp_dict_)
            
    return videos_list, next_tok

In [27]:
'''Hold off on this one until I figure out how to tie in google trends. 
Probably we want only comments from peak viewership plus marks at half peak on either side. 
Right now, most relevant /recent comments are all stupid meme stuff, e.g. who is watching in 2018'''

#This function will grab 50 comments at a time from the video. 
def youtube_comment_list(video_id, max_results=50, sort_method='viewCount', token=None):
  # Call the search.list method to retrieve results matching the specified query term.
    youtube = build(YOUTUBE_API_SERVICE_NAME, YOUTUBE_API_VERSION,
    developerKey=DEVELOPER_KEY)

  # Call the search.list method to retrieve results matching the specified query term.
    search_response = youtube.commentThreads().list(
        videoId = video_id,
        part='snippet, replies',
        pageToken=token,
        order = 'relevance',
        maxResults=max_results,
      ).execute()
    
    return search_response

In [28]:
psy = youtube_comment_list('9bZkp7q19f0')

psy['items'][1]['snippet']['topLevelComment']['snippet']['textDisplay']

'Sep 24th?'

In [6]:
#Automation
def youtube_search_video_looper(q,max_results=50, 
                                sort_method='viewCount', 
                                token=None, 
                                dl_path='./data/test/', dl_title='iteration'):
    #find relevant stats, create dataframe of results, create running csvs
    q=q,
    max_results=max_results
    sort_method = sort_method
    token = token
    
    #Cool Start to get next token
    videos_list, next_tok = youtube_search_video(q=q,
                                                 max_results=max_results, 
                                                 sort_method=sort_method, 
                                                 token=token)
    df_videos = pd.DataFrame(videos_list)
    df_videos['request_token'] = next_tok #for assessment of run.

    count=1
    while next_tok != 'last_page':
        try:

            videos_list, next_tok = youtube_search_video(q,
                                                         max_results=max_results, 
                                                         sort_method=sort_method, 
                                                         token=next_tok)
            print('    found page ', count)
            df_small_vids = pd.DataFrame(videos_list)
            df_small_vids['request_token'] = next_tok
            df_videos = pd.concat([df_videos, df_small_vids], sort=False)


            #create unique text file to track progress
            now_month = datetime.datetime.now().month
            now_day = datetime.datetime.now().day
            now_hour = datetime.datetime.now().hour
            now_minute = datetime.datetime.now().minute
            output_title = '{}{}{}_{}.{}_{}{}.csv'.format(dl_path, dl_title, count, 
                                                          now_month, now_day, now_hour, now_minute)
            df_videos.to_csv(output_title, index=False)

            count += 1
            time.sleep(2)
        except:
            break
    total_pulls = count
    return df_videos, next_tok, total_pulls
        

## Initial Stats for Data Strategy

In [ ]:
df_mv_viewCount = pd.read_csv('./data/running_video_pull.csv')  #This is my interesting class

In [16]:
df_mv_viewCount['viewCount'] = df_mv_viewCount['viewCount'].astype('int64')
df_mv_viewCount['viewCount'].min()

351378

In [17]:
#it looks like youtube doesn't want to return too many videos like this
#perhaps filtering by viewcount limits the effectiveness of relevance
df_mv_viewCount.shape[0]

325

In [18]:
#Given current shape, this is how many irrelevant videos I could use without 
#creating an inbalanced class
print('     max irrelevant: {}\n    final df size: {}'.format( df_mv_viewCount.shape[0] / .25, df_mv_viewCount.shape[0] / .25+df_mv_viewCount.shape[0]))

     max irrelevant: 1300.0
    final df size: 1625.0


In [25]:
print('    num features: {}'.format(df_videos.shape[1]))

    num features: 24


## Get a bunch of data, see what happens

In [20]:
df_videos, next_tok = youtube_search_video_looper(q='music video',max_results=50, 
                                sort_method='relevance', 
                                token=None, 
                                dl_path='./data/music_videos_relevance/', dl_title='iteration2')

    found page  1
    found page  2
    found page  3
    found page  4
    found page  5
    found page  6
    found page  7
    found page  8
    found page  9
    found page  10
    found page  11
    found page  12
    found page  13
    found page  14
    found page  15
    found page  16
    found page  17
    found page  18


good idea to set up a basic cleaning function for these - probably will do a lot

In [22]:
#df_lyrics_viewCount lyric video search by viewcount
#df_mv_relevance music video search by relevance


In [30]:
df_mv_relevance['viewCount'] = df_mv_relevance['viewCount'].map(lambda x: 0 if x=='xxNoneFoundxx' else x)

In [37]:
df_mv_relevance['viewCount'] = df_mv_relevance['viewCount'].astype('int64')

In [74]:
#duplicates
pd.concat([df_mv_viewCount.loc[:, ['title']],  
           df_mv_relevance.loc[:, ['title']]]).sort_values(by='title').duplicated().sum()/2

56.0

In [76]:
#Less duplicates, retrieved 300 new videos
df_mv_relevance.shape[0] - \
pd.concat([df_mv_viewCount.loc[:, ['title']],  
           df_mv_relevance.loc[:, ['title']]]).sort_values(by='title').duplicated().sum()/2

306.0